In [1]:
#load in relevant modules 
import numpy as np
import matplotlib as plt 
import pandas as pd 
from scipy import stats 

In [4]:
##0 - importing the data 
myfile1='diabetic_data.csv'
df1=pd.read_csv(myfile1)

In [5]:
##1 - cleaning the data 
#dropping values from the dataframe 
print(df1.shape) #show the shape of dataframe 

df1.replace('?',np.nan,inplace=True) #replace ? with NaN 
df1 = df1.replace(r'^\s*$', np.nan, regex=True) #replace empty space with NaN

min50= float(0.5*(df1.shape[0]+1)) #create object with value of 50% row total
df1= df1.dropna(axis=1,thresh=min50) #drop columns with values missing from 50% of rows 
df1.shape #one column dropped 

df1.dropna(axis=0,how='any') #drop any rows with missing values 

#drop columns with 95% same values 
col_heads=list(df1.columns)
df1['age'].value_counts(dropna=False) #returns each value with list of counts 
min95=float(0.95*(df1.shape[0]+1)) #determine 95% of values 
for title in col_heads: #for each title in col_heads 
    mylist= df1[title].value_counts(dropna=False) #make a list of counts for each value found in the column
    for x in mylist: #for each value (count no. of each value in col)
        if x >= min95: #if the value is present in 95% of rows 
            df1.drop(axis=1,columns=title,inplace=True)
            statement= 'dropping column: {}'.format(title)
            print(statement)

(101766, 50)
dropping column: repaglinide
dropping column: nateglinide
dropping column: chlorpropamide
dropping column: acetohexamide
dropping column: tolbutamide
dropping column: acarbose
dropping column: miglitol
dropping column: troglitazone
dropping column: tolazamide
dropping column: examide
dropping column: citoglipton
dropping column: glyburide-metformin
dropping column: glipizide-metformin
dropping column: glimepiride-pioglitazone
dropping column: metformin-rosiglitazone
dropping column: metformin-pioglitazone


In [7]:
#transforming age column to midpoint
newcol=[]
for x in df1['age']:
    x=x.replace('[','')
    x=x.replace(')','')
    x2=x.split('-')
    y=int(x2[0])+5
    newcol.append(y)
df1['age']=newcol

In [9]:
#replace NaN values in diag_1/2/3 
diag_cols=['diag_1','diag_2','diag_3']
for title in diag_cols:
    df1[title]= df1[title].fillna(0,inplace=False)
#df1.dtypes

In [11]:
#numerical and other variables from data frame to list 
col_heads=list(df1.columns)
catlist= df1.select_dtypes(object).columns.values.tolist()
numlist= df1.select_dtypes(np.number).columns.values.tolist()

for col in numlist:
    if col[-2:]=='id':
        numlist.remove(col)
        catlist.append(col)
numlist.remove('patient_nbr')
numlist.remove('discharge_disposition_id')
print('numerical columns are: {}\ncategorical columns are: {}'.format(numlist,catlist))
        

numerical columns are: ['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
categorical columns are: ['race', 'gender', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted', 'encounter_id', 'admission_type_id', 'admission_source_id']


In [13]:
#drop rows with duplicate patient_nbr 
df1 = df1.drop_duplicates(subset='patient_nbr',keep='first')
df1.shape

(71518, 33)

In [16]:
#identify and remove outliers 
df2=df1.copy() #create a copy to identify outliers 
for col in numlist: #for each numerical column... 
    z= np.abs(stats.zscore(df1[col])) #calculate a z score 
    df2[col]=z #define a new column with z scores 
    myindex=df2[(df2[col]>3)].index #index depending on whether or not values deviate > 3 sds from mean 
    df1.drop(myindex,inplace=True) #drop all rows with numerical values > 3 sd from mean 
df1.shape


(64468, 33)

In [22]:
df1.loc[4]

encounter_id                    16680
patient_nbr                  42519267
race                        Caucasian
gender                           Male
age                                45
admission_type_id                   1
discharge_disposition_id            1
admission_source_id                 7
time_in_hospital                    1
payer_code                        NaN
medical_specialty                 NaN
num_lab_procedures                 51
num_procedures                      0
num_medications                     8
number_outpatient                   0
number_emergency                    0
number_inpatient                    0
diag_1                            197
diag_2                            157
diag_3                            250
number_diagnoses                    5
max_glu_serum                    None
A1Cresult                        None
metformin                          No
glimepiride                        No
glipizide                      Steady
glyburide   

In [ ]:
## PART 2 -- DATA EXPLORATION 